# FFF step 1: merge prep

In [1]:
%load_ext autoreload
%autoreload 2
import hippo
from os import environ, system
from pathlib import Path
import json
from mrich import print
import mrich
import shutil
import molparse as mp

## Get BulkDock HIPPO database

In [2]:
target_name = "ZIKA_NS3_helicase"

In [3]:
animal = hippo.HIPPO(target_name, Path(environ["BULK"])/"TARGETS"/target_name/f"{target_name}.sqlite")

 Creating HIPPO animal

name = ZIKA_NS3_helicase

db_path = /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/ZIKA_NS3_helicase/ZIKA_NS3_helicase.sqlite

DEBUG: hippo.Database.__init__()

DEBUG: Database.path = /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/ZIKA_NS3_helicase/ZIKA_NS3_helicase.sqlite

DEBUG: hippo.Database.connect()

DEBUG: sqlite3.version='2.6.0'

 Success  Database connected @ 
/opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/ZIKA_NS3_helicase/ZIKA_NS3_helicase.sqlite!

 Success  Initialised animal HIPPO("ZIKA_NS3_helicase")!

## Define merging hypotheses

In [4]:
animal.tags.unique

{'[Other] ATP site',
 '[Other] Covalent to Cys262',
 '[Other] Crystal contact - not trust',
 '[Other] RNA Cleft',
 '[Other] upload_1 2025-01-30',
 'hits'}

In [5]:
merging_hypotheses = {
    # nickname: tag,
    "RNA-Cleft" : "[Other] RNA Cleft",
}

json.dump(merging_hypotheses, open("merging_hypotheses.json", "wt"))

In [6]:
# rename things in the README for convenience
system(f"sed -i 's/TARGET_NAME/{target_name}/g' ../README.md")
system(f"sed -i 's/HYPOTHESIS_NICKNAME/{list(merging_hypotheses.keys())[0]}/g' ../README.md")

0

## Create subdirectories

In [7]:
for nickname in merging_hypotheses:
    (Path("../fragmenstein") / nickname).mkdir(exist_ok=True, parents=True)
    (Path("../knitwork") / nickname).mkdir(exist_ok=True, parents=True)
    (Path("../syndirella") / nickname).mkdir(exist_ok=True, parents=True)

## Knitwork & Fragmenstein inputs

### Create input SDFs & CSVs

In [8]:
for nickname, tag in merging_hypotheses.items():
    
    poses = animal.poses(tag=tag)
    mrich.h3(nickname)
    mrich.var("poses", poses)

    # knitwork
    out_dir = Path(f"../knitwork/{nickname}")
    poses.to_knitwork(out_dir / f"{nickname}_input.csv", path_root=out_dir, aligned_files_dir="../aligned_files")

    # # fragmenstein
    poses.write_sdf(out_dir / f"{nickname}_hits.sdf")
    shutil.copy(out_dir / f"{nickname}_hits.sdf", Path(f"../fragmenstein/{nickname}") / f"{nickname}_hits.sdf")

    ref_pose = poses[0]
    mrich.var("ref_pose", ref_pose)
    shutil.copy(ref_pose.apo_path, Path(f"../fragmenstein/{nickname}"))

╭───────────╮
│ RNA-Cleft │
╰───────────╯

poses = poses tagged "[Other] RNA Cleft": {P × 11}

out_path = /opt/xchem-fragalysis-2/maxwin/Zika_NS3_Helicase_FFF/knitwork/RNA-Cleft/RNA-Cleft_input.csv

path_root = /opt/xchem-fragalysis-2/maxwin/Zika_NS3_Helicase_FFF/knitwork/RNA-Cleft

aligned_files_dir = ../aligned_files

 DISK  Writing /opt/xchem-fragalysis-2/maxwin/Zika_NS3_Helicase_FFF/knitwork/RNA-Cleft/RNA-Cleft_input.csv...

Output()

 DISK  Writing ../knitwork/RNA-Cleft/RNA-Cleft_hits.sdf...

ref_pose = C9->P9: "7ga1-a"

### Create protonated PDBs for Knitwork

In [9]:
for nickname, tag in merging_hypotheses.items():
    poses = animal.poses(tag=tag)
    for pose in mrich.track(poses):
        path = pose.apo_path
        sys = mp.parse(path,verbosity=False)
        sys.add_hydrogens(pH=7)
        subdir = Path("../aligned_files")/pose.name
        subdir.mkdir(exist_ok=True, parents=True)
        sys.write(subdir / str(path.name).replace('_apo-desolv.pdb', '_apo-desolv-Hs.pdb'), verbosity=1)

Output()

 DISK  Writing ../aligned_files/7ga1-a/7ga1-a_apo-desolv-Hs.pdb...

 Warning  fixer.missingAtoms={<Residue 2 (LYS) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (LYS)>, <Atom 4 (CD)
of chain 0 residue 0 (LYS)>, <Atom 5 (CE) of chain 0 residue 0 (LYS)>, <Atom 6 (NZ) of chain 0 residue 0 (LYS)>], 
<Residue 4 (LYS) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (LYS)>, <Atom 4 (CD) of chain 0 residue 0 (LYS)>, 
<Atom 5 (CE) of chain 0 residue 0 (LYS)>, <Atom 6 (NZ) of chain 0 residue 0 (LYS)>]}!

 DISK  Writing ../aligned_files/8v7u-a/8v7u-a_apo-desolv-Hs.pdb...

 DISK  Writing ../aligned_files/7g9m-a/7g9m-a_apo-desolv-Hs.pdb...

...

 DISK  Writing ../aligned_files/7ga3-a/7ga3-a_apo-desolv-Hs.pdb...

 Warning  fixer.missingAtoms={<Residue 2 (LYS) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (LYS)>, <Atom 4 (CD)
of chain 0 residue 0 (LYS)>, <Atom 5 (CE) of chain 0 residue 0 (LYS)>, <Atom 6 (NZ) of chain 0 residue 0 (LYS)>], 
<Residue 4 (LYS) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (LYS)>, <Atom 4 (CD) of chain 0 residue 0 (LYS)>, 
<Atom 5 (CE) of chain 0 residue 0 (LYS)>, <Atom 6 (NZ) of chain 0 residue 0 (LYS)>]}!

 DISK  Writing ../aligned_files/8v7r-b/8v7r-b_apo-desolv-Hs.pdb...

 DISK  Writing ../aligned_files/7g9y-a/7g9y-a_apo-desolv-Hs.pdb...

 DISK  Writing ../aligned_files/7g9k-a/7g9k-a_apo-desolv-Hs.pdb...

## Backup and close database

In [10]:
animal.db.backup()
animal.db.close()

 DISK  Writing 
/opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/ZIKA_NS3_helicase/ZIKA_NS3_helicase_0AJP90O.sqlite...

DEBUG: Copied 139 of 139 pages...

DEBUG: hippo.Database.close()

Closed connection to /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/ZIKA_NS3_helicase/ZIKA_NS3_helicase.sqlite
